# Predicting Employee Attrition

Installing and Importing Libaries 

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

Importing file (csv)

In [2]:
# Import the treated data
df = pd.read_csv('../dados/df.csv')

# Check the data
print(f"Dataset shape: {df.shape}")
print("\nFirst few rows:")
print(df.head())
print("\nColumn types:")
print(df.dtypes)

Dataset shape: (1470, 33)

First few rows:
   EmployeeNumber  Age Attrition     BusinessTravel  DailyRate  \
0               1   41       Yes      Travel_Rarely       1102   
1               2   49        No  Travel_Frequently        279   
2               4   37       Yes      Travel_Rarely       1373   
3               5   33        No  Travel_Frequently       1392   
4               7   27        No      Travel_Rarely        591   

               Department  DistanceFromHome  Education EducationField  \
0                   Sales                 1          2  Life Sciences   
1  Research & Development                 8          1  Life Sciences   
2  Research & Development                 2          2          Other   
3  Research & Development                 3          4  Life Sciences   
4  Research & Development                 2          1        Medical   

   EnvironmentSatisfaction  ... RelationshipSatisfaction  StockOptionLevel  \
0                        2  ...            

## 3. 

### 3.1. 